# Description
This notebook will experiment with:
- loading of graph classification datasets such as PPI, PROTEINS or ENZYMES
- using GNN for graph classification such as MPNN, GGNN and Graph Nets
Goals:
- how to load and feed the graphs to the models
- how to use GNN models
- compare GNN models
- compare to published results

Most of the experiments will be done in PyTorch/PyTorch Geometric, but some models are implemented in Tensor Flow.

# 0. Models

# 1. Codebase

# 2. Experiments

## 2.1 Load a dataset and use GGNN to classify it (train-test setting)

In [80]:
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset
from torch_scatter import scatter_mean
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

# shuffling dataset
dataset = dataset.shuffle()
#equivalent
#perm = torch.randperm(len(dataset))
#dataset = dataset[perm]

# train-test setting
train_dataset = dataset[:540]
test_dataset = dataset[540:]

print("num_classes:",dataset.num_classes)

# batch feeding
loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
for batch in loader:
    break
    # 32 graphs in each batch
    #batch
    #>>> Batch(x=[1082, 21], edge_index=[2, 4066], y=[32], batch=[1082])
    #print(batch.num_graphs)
    #>>> 32
    
    #print(dir(batch))
    
    # the batch contains batch, a list of where does each node belong 
    #(to whhich graph of the batch does each node belong)
    print(batch.batch)
    #print(batch.test_mask)
    
    #print(dir(batch[0])) # this does not work
    
    # compute avereage node features in the node fimension for each graph individually
    #x = scatter_mean(data.x, data.batch, dim=0)
    #x.size()
    #>>> torch.Size([32, 21])

num_classes: 6


In [106]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import MessagePassing
from torch_geometric.nn.conv.gated_graph_conv import GatedGraphConv
from torch_geometric.nn.glob.glob import global_mean_pool, global_add_pool
import torch.nn as nn

class Net1(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=50, num_layers=2,aggr='mean', bias=True)
        self.fc1 = nn.Linear(50, 20)
        self.fc2 = nn.Linear(20, 6)
        self.global_pool = global_add_pool
        
        

    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        #x = self.fc1(x)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #x = self.pool1(x, batch )
        x = F.log_softmax(x, dim=1)
        #x = torch.argmax(x, dim=1)  # we output softmax to use the nll_loss
        
        return x
    
    
class Net2(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=50, num_layers=2,aggr='add', bias=True)
        self.ggnn2 = GatedGraphConv(out_channels=100, num_layers=2,aggr='mean', bias=True)
        self.fc1 = nn.Linear(100, 20)
        self.fc2 = nn.Linear(20, 6)
        self.global_pool = global_add_pool
        
        

    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        x = self.ggnn2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) 
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        
        #x = self.fc1(x)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #x = self.pool1(x, batch )
        x = F.log_softmax(x, dim=1)
        #x = torch.argmax(x, dim=1)  # we output softmax to use the nll_loss
        
        
        return x

In [107]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()

#print(dir(F))

for epoch in range(100):
    print("epoch ",epoch)
    for batch in loader:
        #print(" batch num graphs: ",batch.num_graphs)
        #print(" batch num nodes: ", len(batch.x))
        
        data = batch.to(device)
        #print("data.y: ",data.y)

        optimizer.zero_grad()
        out = model(data)
        # how does this work?
        #print(" out: ",out, " out.dim: ", out.dim(), " data.y.dim(): ", data.y.dim())
        #out = out.view(-1,1)
        #target = data.y.view(-1,1)
        target = data.y
        #print(" out: ", out,"\n target: ",target)
        # transform target to a one-hot-encoding
        # output the softmax values for each example
        loss = F.nll_loss(out, target)
        # C = 6, 
        # target (N) means N values, where each value 0<= <= C-1=5
        # out = (N,C) N values/rows of C classes/columns
        #print("loss: ",loss)
        loss.backward()
        optimizer.step()
        
    print(loss)
        
    

epoch  0
tensor(1.8054, grad_fn=<NllLossBackward>)
epoch  1
tensor(3.7293, grad_fn=<NllLossBackward>)
epoch  2
tensor(2.3745, grad_fn=<NllLossBackward>)
epoch  3
tensor(1.8006, grad_fn=<NllLossBackward>)
epoch  4
tensor(1.8407, grad_fn=<NllLossBackward>)
epoch  5
tensor(1.9914, grad_fn=<NllLossBackward>)
epoch  6
tensor(1.7778, grad_fn=<NllLossBackward>)
epoch  7
tensor(1.7347, grad_fn=<NllLossBackward>)
epoch  8
tensor(1.6641, grad_fn=<NllLossBackward>)
epoch  9
tensor(1.6718, grad_fn=<NllLossBackward>)
epoch  10
tensor(1.8714, grad_fn=<NllLossBackward>)
epoch  11
tensor(1.7405, grad_fn=<NllLossBackward>)
epoch  12
tensor(1.7233, grad_fn=<NllLossBackward>)
epoch  13
tensor(1.7195, grad_fn=<NllLossBackward>)
epoch  14
tensor(1.7201, grad_fn=<NllLossBackward>)
epoch  15
tensor(1.7054, grad_fn=<NllLossBackward>)
epoch  16
tensor(1.6878, grad_fn=<NllLossBackward>)
epoch  17
tensor(1.6725, grad_fn=<NllLossBackward>)
epoch  18
tensor(1.6757, grad_fn=<NllLossBackward>)
epoch  19
tensor(1.655

In [108]:
model.eval()

print("len(test_dataset): ", len(test_dataset))
loader = DataLoader(test_dataset, batch_size= len(test_dataset), shuffle=True)
for batch in loader:
    print("num graphs: ", batch.num_graphs)
    #_, pred = model(test_dataset).max(dim=1)
    _, pred = model(batch).max(dim=1)
    print(pred)
    print(batch.y)
    
    correct = pred.eq(batch.y).sum().item()
    #acc = correct / test_dataset.sum().item()
    acc = correct / batch.num_graphs
    print('Accuracy: {:.4f}'.format(acc))  

len(test_dataset):  60
num graphs:  60
tensor([4, 4, 4, 1, 4, 3, 3, 1, 4, 4, 1, 4, 1, 4, 3, 1, 3, 1, 1, 1, 4, 1, 1, 4,
        1, 1, 3, 1, 1, 4, 1, 1, 4, 4, 1, 3, 1, 4, 4, 1, 1, 1, 5, 1, 1, 1, 1, 5,
        1, 5, 5, 3, 0, 4, 4, 2, 1, 1, 3, 4])
tensor([2, 2, 5, 1, 4, 3, 0, 0, 4, 4, 1, 4, 0, 4, 1, 0, 3, 5, 4, 1, 4, 5, 4, 4,
        5, 5, 0, 4, 5, 4, 0, 1, 4, 4, 1, 3, 5, 2, 4, 5, 1, 1, 5, 2, 1, 1, 1, 5,
        0, 5, 5, 3, 0, 4, 4, 2, 0, 5, 3, 4])
Accuracy: 0.5833


### Results:
1. trained a graph classification model (using global_add_pool)

### Pending:
- review nll_loss (it is negative now, data.y needs to be in one-hot-encoding?)

## 2.2 Load a dataset and use MPNN to classify it (train-validation-test setting)

## 2.3 Load a dataset and use MetaLayer to classify it (train-validation-test setting)

## 2.4 Implement the F1 scores and Accuracy 

## 2.5 Implement a train-val-test setting